In [1]:
import warnings
warnings.filterwarnings('ignore')
from glob import glob
dog_filepaths = glob('data/dog/*.jpg')
dog_filepaths

['data/dog/294.jpg',
 'data/dog/4.jpg',
 'data/dog/430.jpg',
 'data/dog/194.jpg',
 'data/dog/321.jpg',
 'data/dog/399.jpg',
 'data/dog/130.jpg',
 'data/dog/86.jpg',
 'data/dog/24.jpg',
 'data/dog/318.jpg',
 'data/dog/213.jpg',
 'data/dog/154.jpg',
 'data/dog/38.jpg',
 'data/dog/408.jpg',
 'data/dog/393.jpg',
 'data/dog/88.jpg',
 'data/dog/403.jpg',
 'data/dog/37.jpg',
 'data/dog/146.jpg',
 'data/dog/91.jpg',
 'data/dog/332.jpg',
 'data/dog/172.jpg',
 'data/dog/220.jpg',
 'data/dog/54.jpg',
 'data/dog/3.jpg',
 'data/dog/129.jpg',
 'data/dog/292.jpg',
 'data/dog/188.jpg',
 'data/dog/142.jpg',
 'data/dog/17.jpg',
 'data/dog/80.jpg',
 'data/dog/439.jpg',
 'data/dog/244.jpg',
 'data/dog/296.jpg',
 'data/dog/68.jpg',
 'data/dog/368.jpg',
 'data/dog/41.jpg',
 'data/dog/83.jpg',
 'data/dog/364.jpg',
 'data/dog/73.jpg',
 'data/dog/30.jpg',
 'data/dog/211.jpg',
 'data/dog/69.jpg',
 'data/dog/20.jpg',
 'data/dog/39.jpg',
 'data/dog/149.jpg',
 'data/dog/1.jpg',
 'data/dog/208.jpg',
 'data/dog/58.j

In [2]:
cat_filepaths = glob('data/cat/*.jpg')
cat_filepaths

['data/cat/135.jpg',
 'data/cat/262.jpg',
 'data/cat/4.jpg',
 'data/cat/74.jpg',
 'data/cat/130.jpg',
 'data/cat/86.jpg',
 'data/cat/24.jpg',
 'data/cat/248.jpg',
 'data/cat/62.jpg',
 'data/cat/213.jpg',
 'data/cat/154.jpg',
 'data/cat/261.jpg',
 'data/cat/121.jpg',
 'data/cat/38.jpg',
 'data/cat/311.jpg',
 'data/cat/99.jpg',
 'data/cat/88.jpg',
 'data/cat/37.jpg',
 'data/cat/228.jpg',
 'data/cat/146.jpg',
 'data/cat/284.jpg',
 'data/cat/91.jpg',
 'data/cat/263.jpg',
 'data/cat/220.jpg',
 'data/cat/54.jpg',
 'data/cat/118.jpg',
 'data/cat/3.jpg',
 'data/cat/129.jpg',
 'data/cat/120.jpg',
 'data/cat/188.jpg',
 'data/cat/142.jpg',
 'data/cat/17.jpg',
 'data/cat/80.jpg',
 'data/cat/296.jpg',
 'data/cat/312.jpg',
 'data/cat/68.jpg',
 'data/cat/41.jpg',
 'data/cat/108.jpg',
 'data/cat/81.jpg',
 'data/cat/83.jpg',
 'data/cat/278.jpg',
 'data/cat/73.jpg',
 'data/cat/297.jpg',
 'data/cat/30.jpg',
 'data/cat/211.jpg',
 'data/cat/69.jpg',
 'data/cat/96.jpg',
 'data/cat/20.jpg',
 'data/cat/39.jpg

In [3]:
from PIL import Image
import numpy as np
import chainer

In [4]:
imgArray = []
x, t = [], []
dog_t = np.array(0, 'i')
cat_t = np.array(1, 'i')

for dog_filepath, cat_filepath in zip(dog_filepaths, cat_filepaths):
    dog_img = Image.open(dog_filepath)
    cat_img = Image.open(cat_filepath)
    '''
    dog_img = np.array(dog_img, 'f')
    dog_img = dog_img.transpose((2, 0, 1))
    cat_img = np.array(cat_img, 'f')
    cat_img = cat_img.transpose((2, 0, 1))
    '''
    dog_img = chainer.links.model.vision.vgg.prepare(dog_img, size=(224, 224))
    #dog_img = dog_img[np.newaxis] # batch size
    cat_img = chainer.links.model.vision.vgg.prepare(cat_img, size=(224, 224))
    #cat_img = cat_img[np.newaxis] # batch size
    x.append(dog_img)
    x.append(cat_img)
    t.append(dog_t)
    t.append(cat_t)

In [5]:
dataset = chainer.datasets.TupleDataset(x, t)
len(dataset)

500

In [6]:
import chainer
import chainer.links as L
import chainer.functions as F

In [7]:
class CNN(chainer.Chain):

    def __init__(self, n_mid=100, n_out=2):
        super().__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(in_channels=3, out_channels=6, ksize=3, stride=1, pad=1)
            self.conv2 = L.Convolution2D(in_channels=6, out_channels=12, ksize=3, stride=1, pad=1)
            self.fc1 = L.Linear(None, n_mid)
            self.fc2 = L.Linear(None, n_out)
            #self.bn = L.BatchNormalization(3)

    def __call__(self, x):
        h = x
        h = F.relu(self.conv1(h))
        h = F.max_pooling_2d(h, 3, 1)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, 3, 1)
        h = self.fc1(h)
        h = self.fc2(h)
        return h

In [8]:
class VGG16Model(chainer.Chain):

    def __init__(self, out_size):
        super(VGG16Model, self).__init__(
            base = L.VGG16Layers(),
            fc = L.Linear(None, out_size)
        )

    def __call__(self, x):
        h = self.base(x, layers=['fc7'])
        y = self.fc(h['fc7'])
        return y

In [9]:
import random

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

In [11]:
# CPUとGPU関連のシードをすべて固定
reset_seed(0)

In [12]:
#model = L.Classifier(CNN())
#model = VGG16Model()

In [22]:
# インスタンス化
model = L.Classifier(VGG16Model(out_size=2))
optimizer = chainer.optimizers.Adam(alpha=1e-4)
#optimizer = chainer.optimizers.MomentumSGD()
optimizer.setup(model)
model.predictor.base.disable_update()

#model.base.disable_update()

In [23]:
gpu_id = 0  # 使用したGPUに割り振られているID
model.to_gpu(gpu_id)

In [24]:
# Optimizerの定義とmodelとの紐づけ
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [25]:
batchsize = 32

train = int(len(dataset) * 0.7)
train, test = chainer.datasets.split_dataset_random(dataset, train, seed=1)

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

In [26]:
from chainer import training
from chainer.training import extensions

epoch = 200

updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)

trainer = training.Trainer(updater, (epoch, 'epoch'), out='kadai')

# バリデーション用のデータで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))

# 学習結果の途中を表示する
trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))

# １エポックごとに結果をlogファイルに出力させる
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

In [ ]:
trainer.run()

epoch       main/accuracy  validation/main/accuracy  main/loss   validation/main/loss  elapsed_time
1           0.525568       0.534091                  110.029     2.51321               27.0528       
2           0.477273       0.534091                  2.9056      0.694637              53.7669       
3           0.542614       0.465909                  0.700046    0.739281              80.35         
4           0.519886       0.527841                  0.706049    0.691417              106.893       
5           0.534091       0.465909                  0.719535    0.705824              133.445       
6           0.514205       0.465909                  0.724226    0.711148              159.972       
7           0.511364       0.523864                  0.705593    0.694577              186.662       
8           0.542614       0.534091                  0.705812    0.682165              213.362       
9           0.559659       0.518182                  0.703893    0.691147           

79          0.863636       0.614773                  0.336937    1.43956               2116.61       
80          0.88125        0.54375                   0.298067    2.23547               2141.53       
81          0.852273       0.5625                    0.271341    1.51343               2168.62       
82          0.875          0.589773                  0.259682    1.91904               2195.7        
83          0.917614       0.564773                  0.184634    3.03976               2222.79       
84          0.957386       0.558523                  0.114634    3.99422               2249.91       
85          0.948864       0.605682                  0.11858     3.17134               2276.97       
86          0.928977       0.574432                  0.17363     2.96247               2304.04       
87          0.909091       0.552841                  0.244073    3.43897               2331.15       
88          0.897727       0.577273                  0.217857    3.00187          

In [ ]:
import json
import pandas as pd

In [ ]:
with open('kadai/log') as f:
    result = pd.DataFrame(json.load(f))

In [ ]:
result

In [ ]:
result[['main/loss', 'validation/main/loss']].plot()

In [ ]:
result[['main/accuracy', 'validation/main/accuracy']].plot()